In [1]:
import carla
from slotManagement import *

client = carla.Client('localhost', 2000)
world = client.get_world()

bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()
traffic_manager = client.get_trafficmanager()


In [2]:
actor_list = world.get_actors()
vehicle_list = actor_list.filter('vehicle.*')

print(vehicle_list)
ego_vehicle = any

for car in vehicle_list:
    if str(car.type) == "vehicle.tesla.model3":
        ego_vehicle = car

print(ego_vehicle)

[Actor(id=169, type=vehicle.tesla.model3), Actor(id=170, type=vehicle.tesla.model3)]


In [44]:
class Slot:
    def __init__(self, x, y, z, is_occupied=False, vehicle=None):
        self.loc = carla.Location(x, y, z)
        self.is_occupied = is_occupied
        self.vehicle = vehicle
    
    def set_occupied_vehicle(self, vehicle_occupier):
        self.vehicle = vehicle_occupier
        self.is_occupied = True
    
    def set_loc(self, x, y, z):
        self.loc = carla.Location(x,y,z)

    def get_loc(self):
        return self.loc

    def get_is_occupied(self):
        return self.is_occupied
    
    def free_up_slot(self):
        self.vehicle = None

def initialize_grid(grid_size_x, grid_size_y, slot_size, initial_vehicle):
    slots = [[],[]]
    inital_location = initial_vehicle.get_location()
    inital_slot = Slot(inital_location.x, inital_location.y, inital_location.z, True, initial_vehicle.id)
    slots[1].append(inital_slot)
    data_index = 0

    with open("output.txt", "r") as file:
        data = file.readlines()
    for i in range(grid_size_x):
        for j in range(grid_size_y):
            line = data[data_index].strip()
            match = re.match(r"Location\(x=([-.\d]+), y=([-.\d]+), z=([-.\d]+)\)", line)
            x, y, z = map(float, match.groups())
            # TODO update slots to be on map
            slot = Slot(x, y, z)
            slots[i].append(slot)
    slots[grid_size_x-1].pop()
    return slots

def slot_back_propagation(slots, initial_vehicle):
    new_location = initial_vehicle.get_location()

    for slot in slots[1]:
        old_location = slot.get_loc()
        slot.set_loc(new_location.x, new_location.y, new_location.z)
        new_location = old_location
    return slots

slots = initialize_grid(2, 67, 5.0, ego_vehicle)

print(slots[1][0].get_loc())
print(slots[1][1].get_loc())
print(slots[1][2].get_loc())
print(slots[1][3].get_loc())
print(slots[1][4].get_loc())
print("seperator")

# print(slot_grid[1],[2])

Location(x=356.313965, y=326.610077, z=0.001880)
Location(x=392.470001, y=308.210022, z=0.001626)
Location(x=392.470001, y=308.210022, z=0.001626)
Location(x=392.470001, y=308.210022, z=0.001626)
Location(x=392.470001, y=308.210022, z=0.001626)
seperator
